# 参数配置

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

D:\Anaconda\envs\syftpy\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
D:\Anaconda\envs\syftpy\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
D:\Anaconda\envs\syftpy\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
# import syft as sy
# import copy
# hook = sy.TorchHook(torch)

In [3]:
use_cuda = torch.cuda.is_available()

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [4]:
# # create a couple workers
# bob = sy.VirtualWorker(hook, id="bob")
# alice = sy.VirtualWorker(hook, id="alice")
# secure_worker_a = sy.VirtualWorker(hook, id="secure_worker_a")
# secure_worker_b = sy.VirtualWorker(hook, id="secure_worker_b")

In [5]:
# transforms
train_transforms = transforms.Compose([#transforms.RandomRotation(30),
                                       # transforms.RandomResizedCrop(224),
                                       # transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.5,], [0.5,])]) # mean, std
 

test_transforms = transforms.Compose([#transforms.Resize(255),
                                      #transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.5,], [0.5,])]) # mean, std


# choose the training and test datasets

# federated_train_loader = sy.FederatedDataLoader( # <-- this is now a FederatedDataLoader 
#     datasets.CIFAR100('/home/zhaojia-raoxy/data', train=True, download=True,
#                    transform=train_transforms)
#     .federate((bob, alice)), # <-- NEW: we distribute the dataset across all the workers, it's now a FederatedDataset
#     batch_size=20, shuffle=True, **kwargs)


# federated_test_loader = sy.FederatedDataLoader( # <-- this is now a FederatedDataLoader 
#     datasets.CIFAR100('/home/zhaojia-raoxy/data', train=False, download=True,
#                    transform=test_transforms)
#     .federate((secure_worker_a, secure_worker_b)), # <-- NEW: we distribute the dataset across all the workers, it's now a FederatedDataset
#     batch_size=20, shuffle=False, **kwargs)

In [9]:
federated_train_loader = torch.utils.data.DataLoader( # <-- this is now a FederatedDataLoader 
    datasets.CIFAR100('/home/zhaojia-raoxy/data', train=True, download=True,
                   transform=train_transforms), # <-- NEW: we distribute the dataset across all the workers, it's now a FederatedDataset
    batch_size=20, shuffle=True, **kwargs)


federated_test_loader = torch.utils.data.DataLoader( # <-- this is now a FederatedDataLoader 
    datasets.CIFAR100('/home/zhaojia-raoxy/data', train=False, download=True,
                   transform=test_transforms), # <-- NEW: we distribute the dataset across all the workers, it's now a FederatedDataset
    batch_size=20, shuffle=False, **kwargs)

Files already downloaded and verified
Files already downloaded and verified


# 训练模型
## LeNet

In [10]:
class LeNet(nn.Module):
    def __init__(self, channel=3, hideen=768, num_classes=10):
        super(LeNet, self).__init__()
        act = nn.Sigmoid
        self.body = nn.Sequential(
            nn.Conv2d(channel, 12, kernel_size=5, padding=5 // 2, stride=2),
            act(),
            nn.Conv2d(12, 12, kernel_size=5, padding=5 // 2, stride=2),
            act(),
            nn.Conv2d(12, 12, kernel_size=5, padding=5 // 2, stride=1),
            act(),
        )
        self.fc = nn.Sequential(
            nn.Linear(hideen, num_classes)
        )

    def forward(self, x):
        out = self.body(x)
        out = out.view(out.shape[0], -1)
        out = self.fc(out)
        return out
    
model = LeNet(channel=3, hideen=768, num_classes=100).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.03) # TODO momentum is not supported at the moment

## CNN

In [11]:
# class CNN(nn.Module):
#     def __init__(self, num_class):
#         super(CNN, self).__init__()
#         self.block1 = nn.Sequential(
#             nn.Conv2d(in_channels=3, out_channels=64, kernel_size=5),
#             nn.ReLU(),
#             nn.AdaptiveAvgPool2d(5),
#             nn.Dropout(p=0.5)

#         )
#         self.block2 = nn.Sequential(
#             nn.Conv2d(64, 32, 5, padding=1),
#             nn.ReLU(),
#             nn.MaxPool2d(2),
#         )
#         self.block3 = nn.Sequential(
#             nn.Linear(32, num_class),
#             nn.LogSoftmax(dim=1)
#         )

#     def forward(self, x):
#         x = self.block1(x)
#         x = self.block2(x)
#         x = x.view(x.shape[0], -1)  # torch.Size([128, 32])
#         x = self.block3(x)
#         return x
    
# model = CNN(100).to(device)
# optimizer = optim.SGD(model.parameters(), lr=0.03) # TODO momentum is not supported at the moment

## resnet18

In [12]:
# class Arguments():
#     def __init__(self):
#         self.batch_size = 128
#         self.epochs = 50
#         self.lr = 0.02
#         self.num_class = 5
#         self.save_name = 'yeo_alexnet_lr0.01_epoch50'
#         self.data_train = '/home/raoxy/data/train_yeo'
#         self.data_test = '/home/raoxy/data/test_yeo'
#         self.model_name = '/home/raoxy/model/resnet18-5c106cde.pth'
#         self.save_name = 'yeo_resnet18_lr0.01_epoch50'

# args = Arguments()

# def ResNet_s(args):
#     ''':cvar
#     返回修改好的模型，和冻结好的参数
#     '''
#     from torchvision.models import resnet18, resnet34, resnet50, resnet101, resnet152  # ResNet系列
#     pretrain_model = resnet18(pretrained=False)
#     pretrain_model.fc = nn.Linear(pretrain_model.fc.in_features, 100)  # 将全连接层改为自己想要的分类输出
#     pretrained_dict = torch.load(args.model_name)

#     pretrained_dict.pop('fc.weight')
#     pretrained_dict.pop('fc.bias')

#     model_dict = pretrain_model.state_dict()
#     pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}

#     model_dict.update(pretrained_dict)  # 模型参数列表进行参数更新，加载参数
#     pretrain_model.load_state_dict(model_dict)  # 将满足条件的参数的 requires_grad 属性设置为False

# #     for name, value in pretrain_model.named_parameters():
# #         if (name != 'fc.weight') and (name != 'fc.bias'):
# #             value.requires_grad = False
#     params_conv = filter(lambda p: p.requires_grad, pretrain_model.parameters())  # 要更新的参数在parms_conv当中
#     return pretrain_model, params_conv

# model, params_conv = ResNet_s(args)
# optimizer = optim.SGD(params_conv, lr=0.03) # TODO momentum is not supported at the moment

# 训练

In [13]:
criterion = nn.CrossEntropyLoss().to(device)
loss_train=[]
acc_train=[]
loss2=[]

def train(model, device, federated_train_loader, optimizer, epoch, batch_size):
    global criterion
    model.train()
   
    for batch_idx, (data, target) in enumerate(federated_train_loader): # <-- now it is a distributed dataset
#         model.send(data.location) # <-- NEW: send the model to the right location
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target.long())
#         loss = F.cross_entropy(output, target.long())
#         loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
#         model.get() # <-- NEW: get the model back
        
        if batch_idx % 2000 == 0:
#             loss = loss.get() # <-- NEW: get the loss
            # loss_train.append(loss.item())
            # acc_train.append()
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * batch_size, len(federated_train_loader) * batch_size,
                100. * batch_idx / len(federated_train_loader), loss.item()))

In [14]:
def test(model, device, federated_test_loader, batch_size): 
    global criterion
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in federated_test_loader:
#             model.send(data.location) # <-- NEW: send the model in virtual workers to Trusted Aggregator
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target.long())
#             loss = F.nll_loss(output, target, reduction='sum')
#             model.get()
#             test_loss += loss.get() # sum up batch loss
            test_loss += loss.item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(federated_test_loader)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(federated_test_loader) * batch_size,
        100. * correct / len(federated_test_loader) / batch_size))

In [15]:
for epoch in range(50):
    train(model, device, federated_train_loader, optimizer, epoch, batch_size=20)
    test(model, device, federated_test_loader, batch_size=20)

Train Epoch: 0 [0/50000 (0%)]	Loss: 4.594889
Train Epoch: 0 [40000/50000 (80%)]	Loss: 4.612164


KeyboardInterrupt: 

# 结果保存

In [ ]:
save_name='LeNet-no-fed'

In [ ]:
# 保存模型的权重
torch.save(model.state_dict(), "/home/zhaojia-raoxy/model/out/{}.pt".format(save_name))
print("保存文件：","/home/zhaojia-raoxy/model/out/{}.pt".format(save_name))

# 保存整个模型
torch.save(model, "/home/zhaojia-raoxy/model/out/{}.h5".format(save_name))
print("保存文件：","/home/zhaojia-raoxy/model/out/{}.h5".format(save_name))

In [ ]:
# 分析

In [ ]:
txt='''
Train Epoch: 0 [0/50000 (0%)]	Loss: 4.708676
Train Epoch: 0 [40000/50000 (80%)]	Loss: 4.590685

Test set: Average loss: 4.6056, Accuracy: 95/10000 (1%)

Train Epoch: 1 [0/50000 (0%)]	Loss: 4.604600
Train Epoch: 1 [40000/50000 (80%)]	Loss: 4.602582

Test set: Average loss: 4.6052, Accuracy: 100/10000 (1%)

Train Epoch: 2 [0/50000 (0%)]	Loss: 4.596632
Train Epoch: 2 [40000/50000 (80%)]	Loss: 4.611558

Test set: Average loss: 4.6049, Accuracy: 100/10000 (1%)

Train Epoch: 3 [0/50000 (0%)]	Loss: 4.617929
Train Epoch: 3 [40000/50000 (80%)]	Loss: 4.607579

Test set: Average loss: 4.6039, Accuracy: 96/10000 (1%)

Train Epoch: 4 [0/50000 (0%)]	Loss: 4.605203
Train Epoch: 4 [40000/50000 (80%)]	Loss: 4.605900

Test set: Average loss: 4.5987, Accuracy: 102/10000 (1%)

Train Epoch: 5 [0/50000 (0%)]	Loss: 4.589950
Train Epoch: 5 [40000/50000 (80%)]	Loss: 4.492353

Test set: Average loss: 4.4701, Accuracy: 342/10000 (3%)

Train Epoch: 6 [0/50000 (0%)]	Loss: 4.495493
Train Epoch: 6 [40000/50000 (80%)]	Loss: 3.969273

Test set: Average loss: 4.2379, Accuracy: 572/10000 (6%)

Train Epoch: 7 [0/50000 (0%)]	Loss: 4.229941
Train Epoch: 7 [40000/50000 (80%)]	Loss: 4.037433

Test set: Average loss: 4.0686, Accuracy: 781/10000 (8%)

Train Epoch: 8 [0/50000 (0%)]	Loss: 4.223968
Train Epoch: 8 [40000/50000 (80%)]	Loss: 4.097197

Test set: Average loss: 3.9766, Accuracy: 988/10000 (10%)

Train Epoch: 9 [0/50000 (0%)]	Loss: 3.944783
Train Epoch: 9 [40000/50000 (80%)]	Loss: 3.951657

Test set: Average loss: 3.8865, Accuracy: 1147/10000 (11%)

Train Epoch: 10 [0/50000 (0%)]	Loss: 4.041358
Train Epoch: 10 [40000/50000 (80%)]	Loss: 4.043406

Test set: Average loss: 3.7876, Accuracy: 1301/10000 (13%)

Train Epoch: 11 [0/50000 (0%)]	Loss: 3.518500
Train Epoch: 11 [40000/50000 (80%)]	Loss: 3.735777

Test set: Average loss: 3.6846, Accuracy: 1469/10000 (15%)

Train Epoch: 12 [0/50000 (0%)]	Loss: 3.699314
Train Epoch: 12 [40000/50000 (80%)]	Loss: 3.753798

Test set: Average loss: 3.6119, Accuracy: 1626/10000 (16%)

Train Epoch: 13 [0/50000 (0%)]	Loss: 3.669033
Train Epoch: 13 [40000/50000 (80%)]	Loss: 3.134054

Test set: Average loss: 3.5677, Accuracy: 1732/10000 (17%)

Train Epoch: 14 [0/50000 (0%)]	Loss: 2.734629
Train Epoch: 14 [40000/50000 (80%)]	Loss: 3.647416

Test set: Average loss: 3.5019, Accuracy: 1819/10000 (18%)

Train Epoch: 15 [0/50000 (0%)]	Loss: 3.676202
Train Epoch: 15 [40000/50000 (80%)]	Loss: 3.568512

Test set: Average loss: 3.4599, Accuracy: 1882/10000 (19%)

Train Epoch: 16 [0/50000 (0%)]	Loss: 3.509736
Train Epoch: 16 [40000/50000 (80%)]	Loss: 2.810602

Test set: Average loss: 3.4225, Accuracy: 1972/10000 (20%)

Train Epoch: 17 [0/50000 (0%)]	Loss: 4.025714
Train Epoch: 17 [40000/50000 (80%)]	Loss: 2.831839

Test set: Average loss: 3.3644, Accuracy: 2077/10000 (21%)

Train Epoch: 18 [0/50000 (0%)]	Loss: 2.951268
Train Epoch: 18 [40000/50000 (80%)]	Loss: 3.556488

Test set: Average loss: 3.3326, Accuracy: 2135/10000 (21%)

Train Epoch: 19 [0/50000 (0%)]	Loss: 2.837605
Train Epoch: 19 [40000/50000 (80%)]	Loss: 2.981775

Test set: Average loss: 3.3058, Accuracy: 2175/10000 (22%)

Train Epoch: 20 [0/50000 (0%)]	Loss: 3.772324
Train Epoch: 20 [40000/50000 (80%)]	Loss: 3.295967

Test set: Average loss: 3.3042, Accuracy: 2178/10000 (22%)

Train Epoch: 21 [0/50000 (0%)]	Loss: 3.686811
Train Epoch: 21 [40000/50000 (80%)]	Loss: 2.765081

Test set: Average loss: 3.2492, Accuracy: 2291/10000 (23%)

Train Epoch: 22 [0/50000 (0%)]	Loss: 3.459053
Train Epoch: 22 [40000/50000 (80%)]	Loss: 2.574803

Test set: Average loss: 3.2196, Accuracy: 2356/10000 (24%)

Train Epoch: 23 [0/50000 (0%)]	Loss: 3.288209
Train Epoch: 23 [40000/50000 (80%)]	Loss: 2.942970

Test set: Average loss: 3.1960, Accuracy: 2394/10000 (24%)

Train Epoch: 24 [0/50000 (0%)]	Loss: 3.508090
Train Epoch: 24 [40000/50000 (80%)]	Loss: 2.875761

Test set: Average loss: 3.1837, Accuracy: 2442/10000 (24%)

Train Epoch: 25 [0/50000 (0%)]	Loss: 3.097573
Train Epoch: 25 [40000/50000 (80%)]	Loss: 2.783886

Test set: Average loss: 3.1584, Accuracy: 2482/10000 (25%)

Train Epoch: 26 [0/50000 (0%)]	Loss: 3.077751
Train Epoch: 26 [40000/50000 (80%)]	Loss: 2.863167

Test set: Average loss: 3.1382, Accuracy: 2532/10000 (25%)

Train Epoch: 27 [0/50000 (0%)]	Loss: 3.157701
Train Epoch: 27 [40000/50000 (80%)]	Loss: 3.405468

Test set: Average loss: 3.1314, Accuracy: 2521/10000 (25%)

Train Epoch: 28 [0/50000 (0%)]	Loss: 3.381003
Train Epoch: 28 [40000/50000 (80%)]	Loss: 2.636311

Test set: Average loss: 3.1026, Accuracy: 2578/10000 (26%)

Train Epoch: 29 [0/50000 (0%)]	Loss: 2.735946
Train Epoch: 29 [40000/50000 (80%)]	Loss: 2.728709

Test set: Average loss: 3.0930, Accuracy: 2573/10000 (26%)

Train Epoch: 30 [0/50000 (0%)]	Loss: 3.060843
Train Epoch: 30 [40000/50000 (80%)]	Loss: 2.819065

Test set: Average loss: 3.0722, Accuracy: 2642/10000 (26%)

Train Epoch: 31 [0/50000 (0%)]	Loss: 2.701449
Train Epoch: 31 [40000/50000 (80%)]	Loss: 2.802288

Test set: Average loss: 3.0580, Accuracy: 2698/10000 (27%)

Train Epoch: 32 [0/50000 (0%)]	Loss: 2.812822
Train Epoch: 32 [40000/50000 (80%)]	Loss: 2.769371

Test set: Average loss: 3.0651, Accuracy: 2667/10000 (27%)

Train Epoch: 33 [0/50000 (0%)]	Loss: 2.965749
Train Epoch: 33 [40000/50000 (80%)]	Loss: 2.678886

Test set: Average loss: 3.0527, Accuracy: 2725/10000 (27%)

Train Epoch: 34 [0/50000 (0%)]	Loss: 2.349275
Train Epoch: 34 [40000/50000 (80%)]	Loss: 3.397193

Test set: Average loss: 3.0369, Accuracy: 2759/10000 (28%)

Train Epoch: 35 [0/50000 (0%)]	Loss: 2.528225
Train Epoch: 35 [40000/50000 (80%)]	Loss: 2.322527

Test set: Average loss: 3.0389, Accuracy: 2731/10000 (27%)

Train Epoch: 36 [0/50000 (0%)]	Loss: 2.765345
Train Epoch: 36 [40000/50000 (80%)]	Loss: 2.300882

Test set: Average loss: 3.0304, Accuracy: 2732/10000 (27%)

Train Epoch: 37 [0/50000 (0%)]	Loss: 2.536705
Train Epoch: 37 [40000/50000 (80%)]	Loss: 2.632447

Test set: Average loss: 3.0170, Accuracy: 2782/10000 (28%)

Train Epoch: 38 [0/50000 (0%)]	Loss: 2.408066
Train Epoch: 38 [40000/50000 (80%)]	Loss: 2.649137

Test set: Average loss: 3.0178, Accuracy: 2783/10000 (28%)

Train Epoch: 39 [0/50000 (0%)]	Loss: 2.461593
Train Epoch: 39 [40000/50000 (80%)]	Loss: 2.682473

Test set: Average loss: 3.0040, Accuracy: 2799/10000 (28%)

Train Epoch: 40 [0/50000 (0%)]	Loss: 2.265219
Train Epoch: 40 [40000/50000 (80%)]	Loss: 2.050575

Test set: Average loss: 2.9966, Accuracy: 2801/10000 (28%)

Train Epoch: 41 [0/50000 (0%)]	Loss: 2.827886
Train Epoch: 41 [40000/50000 (80%)]	Loss: 2.499863

Test set: Average loss: 2.9927, Accuracy: 2827/10000 (28%)

Train Epoch: 42 [0/50000 (0%)]	Loss: 2.893034
Train Epoch: 42 [40000/50000 (80%)]	Loss: 2.854383

Test set: Average loss: 2.9855, Accuracy: 2853/10000 (29%)

Train Epoch: 43 [0/50000 (0%)]	Loss: 3.673101
Train Epoch: 43 [40000/50000 (80%)]	Loss: 2.847880

Test set: Average loss: 3.0089, Accuracy: 2800/10000 (28%)

Train Epoch: 44 [0/50000 (0%)]	Loss: 2.480524
Train Epoch: 44 [40000/50000 (80%)]	Loss: 2.260675

Test set: Average loss: 2.9796, Accuracy: 2852/10000 (29%)

Train Epoch: 45 [0/50000 (0%)]	Loss: 3.029518
Train Epoch: 45 [40000/50000 (80%)]	Loss: 3.397951

Test set: Average loss: 2.9897, Accuracy: 2827/10000 (28%)

Train Epoch: 46 [0/50000 (0%)]	Loss: 2.133708
Train Epoch: 46 [40000/50000 (80%)]	Loss: 2.689017

Test set: Average loss: 2.9726, Accuracy: 2873/10000 (29%)

Train Epoch: 47 [0/50000 (0%)]	Loss: 2.948929
Train Epoch: 47 [40000/50000 (80%)]	Loss: 2.023050

Test set: Average loss: 2.9661, Accuracy: 2903/10000 (29%)

Train Epoch: 48 [0/50000 (0%)]	Loss: 2.379103
Train Epoch: 48 [40000/50000 (80%)]	Loss: 2.243538

Test set: Average loss: 2.9785, Accuracy: 2904/10000 (29%)

Train Epoch: 49 [0/50000 (0%)]	Loss: 2.108592
Train Epoch: 49 [40000/50000 (80%)]	Loss: 2.367362

Test set: Average loss: 2.9703, Accuracy: 2857/10000 (29%)
'''

In [ ]:
# loss=[]
# acc=[]
# loss2=[]
#
# for i in txt.split('\n'):
#      if len(i)>0:
#         if '40000/50000 ' in i:
#             loss.append(float(i.split()[-1].strip()))
#         if 'Test' in i:
#             loss2.append(float(i.split()[4].strip().replace(",", "")))
#             acc.append(float(i.split()[-2].split('/')[0])/10000)

In [ ]:
# import matplotlib as mpl
# from matplotlib import pyplot as plt
#
# mpl.use('nbAgg')
# mpl.style.use('seaborn-darkgrid')
# import numpy as np
# def plotP(test_loss, train_loss, train_acc_list, test_acc_list):
#     plt.figure(figsize=(6, 8))
#     x = np.linspace(0,  len(train_loss))
#     y = np.linspace(0, len(test_acc_list))
#     plt.subplot(2, 1, 1)
#     plt.plot(x, train_loss, label="train_loss")
#     plt.plot(x, test_loss, label="test_loss")
#
#     plt.xlabel("epoch")
#     plt.ylabel("loss")
#     plt.legend()
#     plt.subplot(2, 1, 2)
#     plt.plot(y, test_acc_list, label="test_acc")
#     plt.xlabel("epoch")
#     plt.ylabel("acc")
#     plt.legend()
#     plt.show()

# plotP(loss2, loss, [], acc)